In [18]:
import pandas as pd

ENERGY_DATASET_PATH = "../dataset/raw/energy_dataset.csv"
WEATHER_DATASET_PATH = "../dataset/raw/weather_features.csv"

energy_dataset = pd.read_csv(ENERGY_DATASET_PATH)
weather_dataset = pd.read_csv(WEATHER_DATASET_PATH)

## Combination and Drop of Columns

In [19]:
weather = weather_dataset.drop(['temp_min', 'temp_max', 'pressure', 'humidity', 'wind_deg', 'rain_1h', 'rain_3h', 'snow_3h', 'weather_id'], axis=1)
print(weather.describe())   

                temp    wind_speed     clouds_all
count  178396.000000  178396.00000  178396.000000
mean      289.618605       2.47056      25.073292
std         8.026199       2.09591      30.774129
min       262.240000       0.00000       0.000000
25%       283.670000       1.00000       0.000000
50%       289.150000       2.00000      20.000000
75%       295.150000       4.00000      40.000000
max       315.600000     133.00000     100.000000


In [20]:
FOSSIL_FUELS_COLUMN_NAME = "fossil_fuels"
WINDPOWER_COLUMN_NAME = "windpower"
SOLARPOWER_COLUMN_NAME = "solarpower"
OTHER_GREEN_ENERGY_COLUMN_NAME = "other_green_energy"

# Load the dataset
df = energy_dataset

# Combine fossil fuel-based generation
fossil_fuels = [
    "generation fossil brown coal/lignite", "generation fossil coal-derived gas",
    "generation fossil gas", "generation fossil hard coal", "generation fossil oil",
    "generation fossil oil shale", "generation fossil peat"
]
df[FOSSIL_FUELS_COLUMN_NAME] = df[fossil_fuels].sum(axis=1)

# Rename and aggregate renewable sources
df[WINDPOWER_COLUMN_NAME] = df[["generation wind offshore", "generation wind onshore"]].sum(axis=1)
# df["windpower forecast"] = df[["forecast wind offshore eday ahead", "forecast wind onshore day ahead"]].sum(axis=1)
df[SOLARPOWER_COLUMN_NAME] = df["generation solar"]
# df["solarpower forecast"] = df["forecast solar day ahead"]

# Aggregate other green energy sources (excluding wind and solar)
green_energy_sources = [
    "generation biomass", "generation geothermal", "generation hydro pumped storage aggregated",
    "generation hydro run-of-river and poundage", "generation hydro water reservoir",
    "generation marine", "generation other", "generation other renewable", "generation waste"
]
df[OTHER_GREEN_ENERGY_COLUMN_NAME] = df[green_energy_sources].sum(axis=1)

# Select relevant columns
columns_to_keep = [
    "time", FOSSIL_FUELS_COLUMN_NAME, WINDPOWER_COLUMN_NAME, SOLARPOWER_COLUMN_NAME,
    OTHER_GREEN_ENERGY_COLUMN_NAME, "total load actual", "price day ahead", "price actual"
]
df = df[columns_to_keep]

# rename space-containing columns
df.rename(columns={"total load actual": "total_load_actual", "price day ahead": "price_day_ahead", "price actual": "price_actual"}, 
          inplace=True)

print(df.head())

                        time  fossil_fuels  windpower  solarpower  \
0  2015-01-01 00:00:00+01:00       10156.0     6378.0        49.0   
1  2015-01-01 01:00:00+01:00       10437.0     5890.0        50.0   
2  2015-01-01 02:00:00+01:00        9918.0     5461.0        50.0   
3  2015-01-01 03:00:00+01:00        8859.0     5238.0        50.0   
4  2015-01-01 04:00:00+01:00        8313.0     4935.0        42.0   

   other_green_energy  total_load_actual  price_day_ahead  price_actual  
0              3709.0            25385.0            50.10         65.41  
1              3425.0            24382.0            48.10         64.92  
2              3104.0            22734.0            47.33         64.48  
3              2475.0            21286.0            42.27         59.32  
4              2407.0            20264.0            38.41         56.04  


C:\Users\Andreas\AppData\Local\Temp\ipykernel_10716\1575167087.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"total load actual": "total_load_actual", "price day ahead": "price_day_ahead", "price actual": "price_actual"},


In [28]:
# format time column
df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%d %H:%M:%S%z", utc=True)
df["time"] = df["time"].dt.tz_convert(None).astype('datetime64[ns]')

# make day-of-week and hour-of-day columns
df["day_of_week"] = df["time"].dt.dayofweek
df["hour_of_day"] = df["time"].dt.hour

C:\Users\Andreas\AppData\Local\Temp\ipykernel_10716\315938985.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%d %H:%M:%S%z", utc=True)
C:\Users\Andreas\AppData\Local\Temp\ipykernel_10716\315938985.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["time"] = df["time"].dt.tz_convert(None).astype('datetime64[ns]')
C:\Users\Andreas\AppData\Local\Temp\ipykernel_10716\315938985.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [29]:
# Save the processed dataset
df.to_csv("../dataset/processed/simplified_dataset.csv", index=False)

# Display the first few rows
print(df.head())



                 time  fossil_fuels  windpower  solarpower  \
0 2014-12-31 23:00:00       10156.0     6378.0        49.0   
1 2015-01-01 00:00:00       10437.0     5890.0        50.0   
2 2015-01-01 01:00:00        9918.0     5461.0        50.0   
3 2015-01-01 02:00:00        8859.0     5238.0        50.0   
4 2015-01-01 03:00:00        8313.0     4935.0        42.0   

   other_green_energy  total_load_actual  price_day_ahead  price_actual  \
0              3709.0            25385.0            50.10         65.41   
1              3425.0            24382.0            48.10         64.92   
2              3104.0            22734.0            47.33         64.48   
3              2475.0            21286.0            42.27         59.32   
4              2407.0            20264.0            38.41         56.04   

   day_of_week  hour_of_day  
0            2           23  
1            3            0  
2            3            1  
3            3            2  
4            3            

## Clean data
There are columns with missing values, so we should interpolate them.

In [31]:
import numpy as np

# set zero values to NaN
df = pd.read_csv("../dataset/processed/simplified_dataset.csv")
df = df.replace(0.0, np.nan)

# count the number of missing values
print(df.isnull().sum())


# interpolate using pandas
df = df.interpolate()

df.to_csv("../dataset/processed/interpolated_dataset.csv", index=False)


time                     0
fossil_fuels            18
windpower               21
solarpower              21
other_green_energy      21
total_load_actual       36
price_day_ahead          0
price_actual             0
day_of_week           5015
hour_of_day           1461
dtype: int64


C:\Users\Andreas\AppData\Local\Temp\ipykernel_10716\1527132032.py:12: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.interpolate()


## Dataset split
We will split the dataset into training, validation, and test datasets using a 80/10/10 ratio.

In [32]:
import os

df = pd.read_csv("../dataset/processed/interpolated_dataset.csv")

# Calculate lengths for train and test sets
train_len = int(len(df) * 0.8)
test_len = len(df) - train_len

# Split the dataset
train, test = df[:train_len], df[train_len:]

# Create directories if they do not exist
os.makedirs("../dataset/processed/train", exist_ok=True)
os.makedirs("../dataset/processed/test", exist_ok=True)

# Save the datasets
train.to_csv("../dataset/processed/train/train.csv", index=False)
test.to_csv("../dataset/processed/test/test.csv", index=False)